## Qualitative Analysis for Event Log with Time-Based Discount

Implemented decision points with guards:

- Request Manager or Standard Approval:
    - Request Manager Approval if total_price >= 800
    - Request Standard Approval if total_price <= 1000
- Manager Rejection or Approval:
    - Manager Rejection if total_price >= 400 and random_exponential(100, 200) > 120
    - Otherwise Manager Approval
- Standard Rejection or Approval:
    - Standard Rejection if total_price >= 400 and random_exponential(100, 200) > 120
    - Otherwise Standard Approval
- Send Order to Vendor or Send Order to Vendor with Discount:
    - Send Order to Vendor if the delay since the previous event is smaller than some threshold
    - Send Order to Vendor with Discount if the delay since the previous event is larger than some threshold
    - This threshold depends on whether or not the order was approved by a manager (i.e., the previous event name)

In [ ]:
import os
from exdpn.util import import_log
from exdpn.data_petri_net import data_petri_net
from exdpn.guards import ML_Technique
from exdpn.util import extend_event_log_with_preceding_event_delay

#### First, look which machine learning guard has the highest performance for a decision point.

In [ ]:
event_log_time_based_discount = import_log(os.path.join(os.getcwd(), '..', 'datasets', 'p2p_with_time_based_discount.xes'))
extend_event_log_with_preceding_event_delay(event_log_time_based_discount, 'delay')
dpn_time_based_discount = data_petri_net.Data_Petri_Net(event_log = event_log_time_based_discount, 
                                                  event_level_attributes = ['item_category','item_id','item_amount','supplier','total_price','delay'],
                                                  hyperparameters={ML_Technique.NN: {'hidden_layer_sizes': (10, 10)},
                                                                        ML_Technique.DT: {'min_samples_split': 0.1,
                                                                                          'min_samples_leaf': 0.1,
                                                                                          'ccp_alpha': 0.1},
                                                                        ML_Technique.LR: {"C": 0.5},
                                                                        ML_Technique.SVM: {"C": 0.5}},
                                                  verbose = False)
# In the above, we have changed the `ccp_alpha` hyperparameter for the decision tree guard.
# This is done since the target class for the "Send Order to Vendor or Send Order to Vendor with Discount"
# decision point is quite unbalanced. The DT classifier would be too general for this decision point
# using the standard hyperparameters

decision_points = list(dpn_time_based_discount.decision_points)
decision_points

In [ ]:
print("Decision point: ", decision_points[0])
dpn_time_based_discount.guard_manager_per_place[decision_points[0]].get_comparison_plot()
best_guard = dpn_time_based_discount.get_guard_at_place(decision_points[0])
print("Best guard:", best_guard)

In [ ]:
print("Decision point: ", decision_points[1])
dpn_time_based_discount.guard_manager_per_place[decision_points[1]].get_comparison_plot()
best_guard = dpn_time_based_discount.get_guard_at_place(decision_points[1])
print("Best guard:", best_guard)

In [ ]:
print("Decision point: ", decision_points[2])
dpn_time_based_discount.guard_manager_per_place[decision_points[2]].get_comparison_plot()
best_guard = dpn_time_based_discount.get_guard_at_place(decision_points[2])
print("Best guard:", best_guard)

In [ ]:
print("Decision point: ", decision_points[3])
dpn_time_based_discount.guard_manager_per_place[decision_points[3]].get_comparison_plot()
best_guard = dpn_time_based_discount.get_guard_at_place(decision_points[3])
print("Best guard:", best_guard)

### Decision Tree

In [ ]:
print("Decision point: ", decision_points[0])
dt_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[0]].guards_list[ML_Technique.DT]
if dt_guard.is_explainable():
    dt_guard.get_explainable_representation()

In [ ]:
print("Decision point: ", decision_points[1])
dt_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[1]].guards_list[ML_Technique.DT]
if dt_guard.is_explainable():
    dt_guard.get_explainable_representation()

In [ ]:
print("Decision point: ", decision_points[1])
dt_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[1]].guards_list[ML_Technique.DT]
if dt_guard.is_explainable():
    dt_guard.get_explainable_representation()

In [ ]:
print("Decision point: ", decision_points[3])
dt_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[3]].guards_list[ML_Technique.DT]
if dt_guard.is_explainable():
    dt_guard.get_explainable_representation()

### Neural Network

In [ ]:
print("Decision point: ", decision_points[0])
nn_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[0]].guards_list[ML_Technique.NN]
if nn_guard.is_explainable():
    # use sample of test data to speed up computation of explainable representation
    sampled_test_data = dpn_time_based_discount.guard_manager_per_place[decision_points[0]].X_test.sample(n = min(100, len(dpn_time_based_discount.guard_manager_per_place[decision_points[0]].X_test)))
    nn_guard.get_explainable_representation(sampled_test_data)

In [ ]:
print("Decision point: ", decision_points[1])
nn_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[1]].guards_list[ML_Technique.NN]
if nn_guard.is_explainable():
    # use sample of test data to speed up computation of explainable representation
    sampled_test_data = dpn_time_based_discount.guard_manager_per_place[decision_points[1]].X_test.sample(n = min(100, len(dpn_time_based_discount.guard_manager_per_place[decision_points[1]].X_test)))
    nn_guard.get_explainable_representation(sampled_test_data)

In [ ]:
print("Decision point: ", decision_points[2])
nn_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[2]].guards_list[ML_Technique.NN]
if nn_guard.is_explainable():
    # use sample of test data to speed up computation of explainable representation
    sampled_test_data = dpn_time_based_discount.guard_manager_per_place[decision_points[2]].X_test.sample(n = min(100, len(dpn_time_based_discount.guard_manager_per_place[decision_points[2]].X_test)))
    nn_guard.get_explainable_representation(sampled_test_data)

In [ ]:
print("Decision point: ", decision_points[3])
nn_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[3]].guards_list[ML_Technique.NN]
if nn_guard.is_explainable():
    # use sample of test data to speed up computation of explainable representation
    sampled_test_data = dpn_time_based_discount.guard_manager_per_place[decision_points[3]].X_test.sample(n = min(100, len(dpn_time_based_discount.guard_manager_per_place[decision_points[3]].X_test)))
    nn_guard.get_explainable_representation(sampled_test_data)

### Support Vector Machine

In [ ]:
print("Decision point: ", decision_points[0])
svm_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[0]].guards_list[ML_Technique.SVM]
if svm_guard.is_explainable():
    svm_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[0]].X_test)

In [ ]:
print("Decision point: ", decision_points[1])
svm_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[1]].guards_list[ML_Technique.SVM]
if svm_guard.is_explainable():
    svm_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[1]].X_test)

In [ ]:
print("Decision point: ", decision_points[2])
svm_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[2]].guards_list[ML_Technique.SVM]
if svm_guard.is_explainable():
    svm_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[2]].X_test)

In [ ]:
print("Decision point: ", decision_points[3])
svm_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[3]].guards_list[ML_Technique.SVM]
if svm_guard.is_explainable():
    svm_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[3]].X_test)

### Logistic Regression

In [ ]:
print("Decision point: ", decision_points[0])
lr_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[0]].guards_list[ML_Technique.LR]
if lr_guard.is_explainable():
    lr_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[0]].X_test)

In [ ]:
print("Decision point: ", decision_points[1])
lr_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[1]].guards_list[ML_Technique.LR]
if lr_guard.is_explainable():
    lr_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[1]].X_test)

In [ ]:
print("Decision point: ", decision_points[2])
lr_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[2]].guards_list[ML_Technique.LR]
if lr_guard.is_explainable():
    lr_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[2]].X_test)

In [ ]:
print("Decision point: ", decision_points[3])
lr_guard = dpn_time_based_discount.guard_manager_per_place[decision_points[3]].guards_list[ML_Technique.LR]
if lr_guard.is_explainable():
    lr_guard.get_explainable_representation(dpn_time_based_discount.guard_manager_per_place[decision_points[3]].X_test)

### Summary Event Log with Time-Based Discount: - needs to be update when implementations are all done -

- Request Manager or Standard Approval (similar to Event Log without randomness):  
Only the Decision Tree guard models the true underlying guard. The split to classify the date samples is total_price <= 897. For the other three machine learning techniques the feature attribute item_amount has a huge impact on the model prediction, this attribute is correlated with total_price.

- Manager Rejection or Approval (similar to Event Log without randomness):  
All machine learning techniques have problems modeling the true underlying guard. We can see that the Support Vector Machine guard again uses item_amout for its prediction. Logistic Regression guard seems to pick up noise instead of the true underlying guard. But it is possible that the features with the most impact correlate with the total_price. For the Neural Network guard all feature attributes only have a very small impact on the model prediction, thus the plot is empty.

- Standard Rejection or Approval (similar to Event Log without randomness):  
Again, all techniques struggle to model the true underlying guard. Only for the Logistic Regression guard total_price is the feature attribute with the highest impact. The Support Vector Machine guard seems to pick up noise instead of the true underlying guard. But it could also be possible that the features with the most impact correlate with the total_price (e.g., maybe "Office Supplies" often have a total_price >= or < 500). For the Neural Network guard all feature attributes only have a very small impact on the model prediction, thus the plot is empty.

- Send Order to Vendor or Send Order to Vendor with Discount:  
All models show that the event-level delay attribute has the greatest impact on the classification. For all models but the decision tree, we can observe that the previous path in the case (i.e., if the order went through the standard or the manager approval "route") was also among the high impact attributes. This makes sense since the concrete delay threshold at which an order with a discount is issued depends on this information. It is to be noted that the default hyperparameters for the decision tree classifier lead to an underfitting model due to pruning.

Overall, all machine learning techniques have their problems modeling the true underlying behavior of the guard if the decisions are partly random. But we can also see, that for example the Decision Tree is at advantage for modeling a binary numeric decision with some overlapping, while the other three techniques are able to detect some correlation between other feature attributes and the feature attribute used in the true guard. All four machine learning guards achieve high performance scores measures using F1-score as seen in the comparision plots.
